In [1]:
ver = 2

In [2]:
import re
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import transformers

from sklearn.metrics import accuracy_score

# os.environ["CUDA_VISIBLE_DEVICES"]="0"

tf.random.set_seed(0)
np.random.seed(0)

In [3]:
data_path = '../data/'

train_origin = pd.read_csv(data_path+'train_data.csv')
test_origin = pd.read_csv(data_path+'test_data.csv')
topic_dict_origin = pd.read_csv(data_path+'topic_dict.csv')
sample_submission_origin = pd.read_csv(data_path+'sample_submission.csv')

In [4]:
train = train_origin.copy()
test = test_origin.copy()
topic_dict = topic_dict_origin.copy()
sample_submission = sample_submission_origin.copy()

In [5]:
tokenizer = transformers.BertTokenizerFast.from_pretrained(
    'kykim/bert-kor-base',
    cache_dir = '../tokenizer/bert-base-multilingual-cased',
    do_lower_case = False
)

In [6]:
data_config = {}

# result = []
# for i in tqdm(range(len(X_train_encoded['attention_mask']))):
#     result.append(sum(X_train_encoded['attention_mask'][i].numpy()))

data_config['max_length'] = 26

In [7]:
def cleaning_text(text):
    text = re.sub('[^a-z가-힣]', ' ', text.lower())
    text = re.sub('[\s]+', ' ', text)
    return text

In [8]:
X_train = train['title'].apply(cleaning_text)
X_train = X_train.values.tolist()

X_train_encoded = tokenizer(
    X_train,
    padding = 'max_length',
    truncation = True,
    max_length = data_config['max_length'],
    return_tensors = 'tf'
)

In [9]:
y_train = train['topic_idx'].values

In [10]:
bert_model = transformers.TFBertModel.from_pretrained(
    'kykim/bert-kor-base', 
    cache_dir = '../model/kykim/bert-kor-base/cache/',
    output_hidden_states=False,
    output_attentions=False,
    use_cache = False
)

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at kykim/bert-kor-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [11]:
def set_model(learning_rate=3e-5, print_summary=False):

    input_ids = Input(batch_shape=(None, data_config['max_length']), dtype=tf.int32, name='input_ids')
    input_masks = Input(batch_shape=(None, data_config['max_length']), dtype=tf.int32, name='attention_masks')
    bert_outputs = bert_model([input_ids, input_masks])['pooler_output']
    dropout_0 = Dropout(0.1)(bert_outputs)
    dense_0 = Dense(64, activation='relu', name='dense_0')(dropout_0)
    dropout_1 = Dropout(0.1)(dense_0)
    dense_1 = Dense(32, activation='relu', name='dense_1')(dropout_1)
    dropout_2 = Dropout(0.1)(dense_1)
    outputs = Dense(7, activation='softmax', name='outputs')(dropout_2)

    model = Model(
        inputs = [input_ids, input_masks],
        outputs = outputs,
        name = 'Bert_Classification'
    )

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        loss = 'sparse_categorical_crossentropy',
        optimizer = optimizer, 
        metrics = ['accuracy']
    )

    if print_summary:
        model.summary(line_length=150)
    
    return model

In [12]:
model = set_model(learning_rate=9e-5)
model.layers[2].trainable = False
for i in range(len(model.layers[2].weights)):
    model.layers[2].weights[i]._trainable = False
model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f4a06a45110> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f4a06a45110> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Model: "Bert_Classification"
__________________________________________________________________________

In [13]:
history = model.fit(
    x = [X_train_encoded['input_ids'][:10000], X_train_encoded['attention_mask'][:10000]], 
    y = y_train[:10000],
    epochs = 1,
    batch_size = 64
)

157/157 [==============================] - 14s 70ms/step - loss: 1.9330 - accuracy: 0.1929


In [14]:
model.layers[2].trainable = True

for i in range(len(model.layers[2].weights)):
    model.layers[2].weights[i]._trainable = True
for i in range(133):
    model.layers[2].weights[i]._trainable = False

optimizer = keras.optimizers.Adam(learning_rate=3e-5)
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = optimizer, 
    metrics = ['accuracy']
)

model.summary()

Model: "Bert_Classification"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 26)]         0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 26)]         0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 118297344   input_ids[0][0]                  
                                                                 attention_masks[0][0]            
__________________________________________________________________________________________________
dropout_37 (Dropout)            (None, 768)          0           tf_bert_model[0

In [15]:
early_stop = EarlyStopping(
    monitor = 'val_loss',
    patience = 3
)
model_check_point = ModelCheckpoint(
    filepath = f'../model/bert_kor_base_fine_tuning_{ver}.h5',
    monitor = 'val_accuracy',
    verbose = 1,
    min_delta = 0.02,
    mode = 'max',
    save_best_only = True,
    save_weights_only = True
)
def scheduler(epoch, lr):
    return lr * 0.95
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

history = model.fit(
    x = [X_train_encoded['input_ids'], X_train_encoded['attention_mask']], 
    y = y_train,
    epochs = 20,
    batch_size = 32,
    validation_split = 0.2,
    callbacks = [early_stop, model_check_point, lr_scheduler]
)

Epoch 1/20


1142/1142 [==============================] - 85s 70ms/step - loss: 0.6472 - accuracy: 0.7916 - val_loss: 0.5732 - val_accuracy: 0.8043

Epoch 00001: val_accuracy improved from -inf to 0.80429, saving model to ../model/bert_kor_base_fine_tuning_2.h5
Epoch 2/20
1142/1142 [==============================] - 80s 70ms/step - loss: 0.4640 - accuracy: 0.8511 - val_loss: 0.5146 - val_accuracy: 0.8240

Epoch 00002: val_accuracy improved from 0.80429 to 0.82401, saving model to ../model/bert_kor_base_fine_tuning_2.h5
Epoch 3/20
1142/1142 [==============================] - 79s 69ms/step - loss: 0.4150 - accuracy: 0.8659 - val_loss: 0.5247 - val_accuracy: 0.8213

Epoch 00003: val_accuracy did not improve from 0.82401
Epoch 4/20
1142/1142 [==============================] - 79s 69ms/step - loss: 0.3753 - accuracy: 0.8769 - val_loss: 0.4916 - val_accuracy: 0.8356

Epoch 00004: val_accuracy improved from 0.82401 to 0.83561, saving model to ../model/bert_kor_base_fine_tuning_2.h5
Epoch 5/20
1142/1142 [=

In [16]:
best_model = set_model()
best_model.load_weights(f'../model/bert_kor_base_fine_tuning_{ver}.h5')

In [17]:
y_train_pred = best_model.predict([X_train_encoded['input_ids'], X_train_encoded['attention_mask']])
y_train_pred = np.argmax(y_train_pred, axis=1)

print(f'train accuracy: {accuracy_score(y_train, y_train_pred):.04f}')

train accuracy: 0.9126


In [18]:
X_test = test['title'].apply(cleaning_text)
X_test = X_test.values.tolist()

X_test_encoded = tokenizer(
    X_test,
    padding = 'max_length',
    truncation = True,
    max_length = data_config['max_length'],
    return_tensors = 'tf'
)

In [19]:
y_test_pred = best_model.predict([X_test_encoded['input_ids'], X_test_encoded['attention_mask']])
y_test_pred = np.argmax(y_test_pred, axis=1)

sample_submission['topic_idx'] = y_test_pred

In [20]:
sample_submission.to_csv(f'../submit/bert_kor_base_{ver}.csv', index=False)